In [4]:
import json
import random
from pathlib import Path

# Notebook definition
nb = {
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Generazione dinamica di few-shot\n",
        "\n",
        "Estrae 15 esempi **complessi** e 15 esempi **semplici** da `data/oncomx/dev.json`\n",
        "e li salva in `prompts/few_shot_examples.txt`."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": None,
      "metadata": {},
      "outputs": [],
      "source": [
        "import json\n",
        "import random\n",
        "from pathlib import Path\n",
        "\n",
        "# Parametri\n",
        "DEV_PATH   = Path('data/oncomx/dev.json')\n",
        "OUT_PATH   = Path('prompts/few_shot_examples.txt')\n",
        "N_TOTAL    = 30    # esempi totali\n",
        "N_COMPLEX  = 15    # di questi, quanti complessi\n",
        "COMPLEX_KW = ('JOIN','GROUP','DISTINCT','COUNT','EXISTS','IN (')\n",
        "\n",
        "# Carica tutti gli esempi\n",
        "with DEV_PATH.open(encoding='utf-8') as f:\n",
        "    data = json.load(f)\n",
        "all_ex = []\n",
        "for ex in data:\n",
        "    q   = ex.get('question','').strip().replace('\\n',' ')\n",
        "    sql = ex.get('query', ex.get('sql','')).strip().rstrip(';')\n",
        "    if q and sql:\n",
        "        all_ex.append((q, sql))\n",
        "\n",
        "# Suddividi in complessi e semplici\n",
        "complex_ex = [e for e in all_ex if any(kw in e[1].upper() for kw in COMPLEX_KW)]\n",
        "simple_ex  = [e for e in all_ex if e not in complex_ex]\n",
        "\n",
        "# Campiona a caso\n",
        "random.seed(42)\n",
        "chosen = random.sample(complex_ex, min(N_COMPLEX, len(complex_ex)))\n",
        "remain = N_TOTAL - len(chosen)\n",
        "chosen += random.sample(simple_ex, min(remain, len(simple_ex)))\n",
        "\n",
        "# Scrivi il few-shot\n",
        "OUT_PATH.parent.mkdir(parents=True, exist_ok=True)\n",
        "with OUT_PATH.open('w', encoding='utf-8') as f:\n",
        "    for q, sql in chosen:\n",
        "        f.write(f\"-- Question: {q}\\n-- SQL: {sql}\\n\\n\")\n",
        "\n",
        "print(f\"Written {len(chosen)} examples to {OUT_PATH}\")"
      ]
    }
  ],
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "name": "python",
      "version": "3.x"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 5
}

# Salva il notebook
notebook_path = Path("generate_few_shot.ipynb")
with notebook_path.open("w", encoding="utf-8") as f:
    json.dump(nb, f, indent=2)

print(f"Notebook written to {notebook_path}")


Notebook written to generate_few_shot.ipynb
